<a href="https://colab.research.google.com/github/PerimeterMaison/Assignment-4/blob/main/Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


OBS: CTRL+F ASDF. ALLT SOM HAR ASDF EFTER SIG MÅSTE KORRIGERAS!

/MoMoney

# DAT565 Introduction to Data Science and AI
## 2023-2024, LP2
## Assignment 4: Spam classification using Naïve Bayes
This assignment has three obligatory questions. Questions 4-5 are optional and will not be graded.


The exercise takes place in this notebook environment where you can chose to use Jupyter or Google Colabs. We recommend you use Google Colabs as it will facilitate remote group-work and makes the assignment less technical.

*Tips:*
* You can execute certain Linux shell commands by prefixing the command with a `!`.
* You can insert Markdown cells and code cells. The first you can use for documenting and explaining your results, the second you can use to write code snippets that execute the tasks required.  

In this assignment you will implement a Naïve Bayes classifier in Python that will classify emails into spam and non-spam (“ham”) classes.  Your program should be able to train on a given set of spam and “ham” datasets.

You will work with the datasets available at https://spamassassin.apache.org/old/publiccorpus/. There are three types of files in this location:
-	easy-ham: non-spam messages typically quite easy to differentiate from spam messages.
-	hard-ham: non-spam messages more difficult to differentiate
-	spam: spam messages

**Execute the cell below to download and extract the data into the environment of the notebook -- it will take a few seconds.**

If you chose to use Jupyter notebooks you will have to run the commands in the cell below on your local computer. Note that if you are using Windows, you can instead use [7zip](https://www.7-zip.org/download.html) to decompress the data (you will have to modify the cell below).

**What to submit:**
* Convert the notebook to a PDF file by compiling it, and submit the PDF file.
* Make sure all cells are executed so all your code and its results are included.
* Double-check that the PDF displays correctly before you submit it.

In [1]:
# Downloads and extracts the data. "wget" and "tar" are usually used in Linux
# but can also be used in Google Colab notebook since Google Colab runs on
# Linux.

!wget https://spamassassin.apache.org/old/publiccorpus/20021010_easy_ham.tar.bz2
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_hard_ham.tar.bz2
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_spam.tar.bz2
!tar -xjf 20021010_easy_ham.tar.bz2
!tar -xjf 20021010_hard_ham.tar.bz2
!tar -xjf 20021010_spam.tar.bz2

--2023-11-25 12:18:52--  https://spamassassin.apache.org/old/publiccorpus/20021010_easy_ham.tar.bz2
Resolving spamassassin.apache.org (spamassassin.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to spamassassin.apache.org (spamassassin.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1677144 (1.6M) [application/x-bzip2]
Saving to: ‘20021010_easy_ham.tar.bz2’

20021010_easy_ham.t 100%[===================>]   1.60M  --.-KB/s    in 0.08s   

2023-11-25 12:18:53 (20.2 MB/s) - ‘20021010_easy_ham.tar.bz2’ saved [1677144/1677144]

--2023-11-25 12:18:53--  https://spamassassin.apache.org/old/publiccorpus/20021010_hard_ham.tar.bz2
Resolving spamassassin.apache.org (spamassassin.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to spamassassin.apache.org (spamassassin.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1021126 (997K) [application/x-bzip2]
Saving to: ‘20021010_hard_ha

The data is now in the following three folders: `easy_ham`, `hard_ham`, and `spam`. You can confirm this via the following command:

In [ ]:
!ls -lah

total 4.0M
drwxr-xr-x 1 root root 4.0K Nov 24 20:55 .
drwxr-xr-x 1 root root 4.0K Nov 24 20:53 ..
-rw-r--r-- 1 root root 1.6M Jun 29  2004 20021010_easy_ham.tar.bz2
-rw-r--r-- 1 root root 998K Dec 16  2004 20021010_hard_ham.tar.bz2
-rw-r--r-- 1 root root 1.2M Jun 29  2004 20021010_spam.tar.bz2
drwxr-xr-x 4 root root 4.0K Nov 21 14:21 .config
drwx--x--x 2  500  500 168K Oct 10  2002 easy_ham
drwx--x--x 2 1000 1000  20K Dec 16  2004 hard_ham
drwxr-xr-x 1 root root 4.0K Nov 21 14:24 sample_data
drwxr-xr-x 2  500  500  36K Oct 10  2002 spam


### 1. Preprocessing:
Note that the email files contain a lot of extra information, besides the actual message. Ignore that for now and run on the entire text (in the optional part further down, you can experiment with filtering out the headers and footers).
1.	We don’t want to train and test on the same data (it might help to reflect on **why** ,if you don't recall). Split the spam and ham datasets into a training set and a test set. (`hamtrain`, `spamtrain`, `hamtest`, and `spamtest`). Use `easy_ham` for quesions 1 and 2.


In [ ]:
# Write your pre-processing code here

import os
# import re #Works like a magnifying glass meaning that it is used to find
            #specific/unique words, no need for it right now! ASDF
from sklearn.model_selection import train_test_split

# Function to read files from a directory into a list
def read_email_files(directory):
    files = os.listdir(directory)
    emails = []
    for file in files:
        with open(os.path.join(directory, file), 'r', encoding='latin1') as email_file:
            emails.append(email_file.read())
    return emails

# Assuming the directories are named 'easy_ham' and 'spam' in your current working directory
easy_ham = read_email_files('/content/easy_ham')
spam = read_email_files('/content/spam')

# Split each dataset into training and test sets
hamtrain, hamtest = train_test_split(easy_ham, test_size=0.2, random_state=42)
spamtrain, spamtest = train_test_split(spam, test_size=0.2, random_state=42)

# Now hamtrain, hamtest, spamtrain, and spamtest contain your actual email data


### 2. Write a Python program that:
1.	Uses the four datasets from Question 1 (`hamtrain`, `spamtrain`, `hamtest`, and `spamtest`).
2.	Trains a Naïve Bayes classifier (use the [scikit-learn library](https://scikit-learn.org/stable/)) on `hamtrain` and `spamtrain`, that classifies the test sets and reports True Positive and False Negative rates on the `hamtest` and `spamtest` datasets. You can use `CountVectorizer` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer)) to transform the email texts into vectors. Please note that there are different types of Naïve Bayes Classifiers available in *scikit-learn* ([Documentation here](https://scikit-learn.org/stable/modules/naive_bayes.html)). Here, you will test two of these classifiers that are well suited for this problem:
- Multinomial Naive Bayes
- Bernoulli Naive Bayes.

Please inspect the documentation to ensure input to the classifiers is appropriate before you start coding. You may have to modify your input.

In [5]:
# write your code here

#What does the code do?

# The code reads email data from directories, splits it into training and
# testing datasets, converts the email texts into numerical vectors, and then
# uses these vectors to train and evaluate two types of Naïve Bayes classifiers,
# Multinomial and Bernoulli - by calculating their
# True Positives and False Negatives.

# The reason the email texts are converted to numerical value is because
#  ASDF

# We already have our email data split into training and test sets for both
# 'ham' (non-spam) and 'spam' emails from Task 1. Next step is to convert
# these text emails into a format that our machine learning model
# can understand.
# This is done using CountVectorizer from scikit-learn, which converts text
#  data into a matrix of token counts.


# Needed modules to perform task 2:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import confusion_matrix


# Combines ham and spam datasets
train_emails = hamtrain + spamtrain
test_emails = hamtest + spamtest

# Create labels
train_labels = [0]*len(hamtrain) + [1]*len(spamtrain)
test_labels = [0]*len(hamtest) + [1]*len(spamtest)

# Convert emails to a matrix of token counts
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_emails)
X_test = vectorizer.transform(test_emails)

# Train the Multinomial Naive Bayes model
mnb = MultinomialNB()
mnb.fit(X_train, train_labels)

# Train the Bernoulli Naive Bayes model
bnb = BernoulliNB()
bnb.fit(X_train, train_labels)

# Predictions
mnb_pred = mnb.predict(X_test)
bnb_pred = bnb.predict(X_test)

# Evaluate the models
def evaluate_model(predictions, labels):
    tn, fp, fn, tp = confusion_matrix(labels, predictions).ravel()
    print("True Positives (TP):", tp)
    print("False Negatives (FN):", fn)

print("Multinomial Naive Bayes:")
evaluate_model(mnb_pred, test_labels)

print("\nBernoulli Naive Bayes:")
evaluate_model(bnb_pred, test_labels)

# The explanation for the output produced from this code cell
# (outputs being the TP and  FN) is the following: ASDF


Multinomial Naive Bayes:
True Positives (TP): 72
False Negatives (FN): 29

Bernoulli Naive Bayes:
True Positives (TP): 38
False Negatives (FN): 63


### 3. Run on hard ham:
Run the two models from Question 2 on `spam` versus `hard-ham`, and compare to the `easy-ham` results.

In [ ]:
# code to report results here

### 4.	OPTIONAL - NOT MARKED:
To avoid classification based on common and uninformative words, it is common practice to filter these out.

**a.** Think about why this may be useful. Show a few examples of too common and too uncommon words.

**b.** Use the parameters in *scikit-learn*’s `CountVectorizer` to filter out these words. Update the program from Question 2 and run it on `easy-ham` vs `spam` and `hard-ham` vs `spam`. Report your results.

In [ ]:
# write your code here

### 5. OPTIONAL - NOT MARKED: Further improving performance
Filter out the headers and footers of the emails before you run on them. The format may vary somewhat between emails, which can make this a bit tricky, so perfect filtering is not required. Run your program again and answer the following questions:
- Does the result improve from those obtained in Questions 3 and 4?
- What do you expect would happen if your training set consisted mostly of spam messages, while your test set consisted mostly of ham messages, or vice versa?
- Look at the `fit_prior` parameter. What does this parameter mean? Discuss in what settings it can be helpful (you can also test your hypothesis).

In [ ]:
# write your code here